In [1]:
import numpy as np
import sys
from collections import defaultdict
from gridworld_env import Gridworld
from gridworld_env import Action

In [2]:
NUM_EPISODES = 500_000
GAMMA = 0.95

In [3]:
def generate_episode_with_random_policy(env):
    episode = []
    state = env.reset()
    done = False
    while not done:
        action = np.random.randint(0, env.action_space.n)
        next_state, reward, done, info = env.step(action)
        episode.append((state, action, reward))
        state = next_state
    return episode

In [4]:
def mc_prediction_q(env, num_episodes, generate_episode, gamma=1.0):
    N = defaultdict(lambda: np.zeros(env.action_space.n, dtype=int))
    Q = defaultdict(lambda: np.zeros(env.action_space.n))

    for i_episode in range(1, num_episodes+1):
        # monitor progress
        if i_episode % 1000 == 0:
            print("\rEpisode {}/{}".format(i_episode, num_episodes), end="")
            sys.stdout.flush()
        
        episode = generate_episode(env)
        g = 0.0
        states, _, _ = zip(*episode)
        for step in reversed(range(0,len(episode))):
            (state, action, reward) = episode[step]
            g = reward + gamma * g
            # Is this the first visit? (First-Visit-MC)
            if not state in states[:step]:
                N[state][action] += 1
                Q[state][action] = Q[state][action] + (1/N[state][action])*(g - Q[state][action])
    return Q

In [5]:
env = Gridworld()

In [6]:
episode = generate_episode_with_random_policy(env)
# episode

In [7]:
env.gridworld

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

In [8]:
env.state_transition_matrix

array([[ 0,  1,  4,  0],
       [ 1,  2,  5,  0],
       [ 2,  3,  6,  1],
       [ 3,  3,  7,  2],
       [ 0,  5,  8,  4],
       [ 1,  6,  9,  4],
       [ 2,  7, 10,  5],
       [ 3,  7, 11,  6],
       [ 4,  9, 12,  8],
       [ 5, 10, 13,  8],
       [ 6, 11, 14,  9],
       [ 7, 11, 15, 10],
       [ 8, 13, 12, 12],
       [ 9, 14, 13, 12],
       [10, 15, 14, 13],
       [11, 15, 15, 14]])

In [9]:
Q = mc_prediction_q(env, NUM_EPISODES, generate_episode_with_random_policy, GAMMA)

Episode 500000/500000

In [10]:
Q

defaultdict(<function __main__.mc_prediction_q.<locals>.<lambda>()>,
            {11: array([-13.69428612,  -9.81376499,  -1.        , -12.67047184]),
             12: array([-15.68273573, -13.6870906 , -15.18554731, -15.24515404]),
             5: array([-16.27638422, -14.74814555, -14.71993336, -16.27729394]),
             8: array([-16.27500871, -14.7452153 , -15.19658031, -15.68980547]),
             9: array([-15.7183528 , -12.63791842, -13.71120211, -15.70652665]),
             13: array([-14.71638925,  -9.83524602, -13.67908661, -15.19759253]),
             14: array([-12.67090367,  -1.        ,  -9.80222408, -13.69170729]),
             10: array([-14.73123719,  -9.78892882,  -9.81755556, -14.69511207]),
             6: array([-15.71003863, -13.69121185, -12.6647177 , -15.72746507]),
             7: array([-15.21528   , -13.70422903,  -9.84463601, -14.71016915]),
             3: array([-15.19152243, -15.2509048 , -13.7252471 , -15.72149516]),
             2: array([-15.7184677 

In [11]:
V = np.zeros(env.observation_space.n)
for k,v in Q.items():
    V[k] = np.round(np.max(v),0)
V = V.reshape((4, 4))

In [12]:
V

array([[-16., -16., -15., -14.],
       [-16., -15., -13., -10.],
       [-15., -13., -10.,  -1.],
       [-14., -10.,  -1.,   0.]])